# Tutoriel #4 - Les prénoms en France depuis 1900

Ce notbook est une introduction à :

1. **trois libraries python** facilitant l'analyse de données. 
   - matplotlib : permet de faire des graphiques
   - numpy : permet de faciliter le travail avec des matrices et des vecteurs
   - pandas : offre la possibilité de manipuler des tableaux de données plus complexe (DataFrame)
1. **l'analyse de données**, ici les prénoms donnés aux enfants nés en France de 1900 à 2023 (sources INSEE).
2. **la visualisation sur une carte** des résultats de cette analyse, en utilisant les techniques du GéoWeb et du Mashup.

Nous nous servirons d'un jeu de données sur le nombre de naissances par prénoms, par départements et années pour illustrer l'usage de ces bibliothèques en essayant au passage d'explorer ces données.

> Source des données : [insee - *Prénoms attribués aux enfants nés en France depuis 1900*](https://www.insee.fr/fr/statistiques/8205621?sommaire=8205628)


## Mise en place
Commençons par charger ces trois librairies et définir le mode d'affichage des figures de notre notebook avec l'instruction jupyter `%matplotlib inline`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Lecture du jeu de données et information contextuelles

Nous pouvons charger les données avec pandas. La fonction 'read_csv' permet de charger les données nous psécifions juste le séparateur de champs en l'occurence `;`. Nous pouvons afficher les noms et le type des colonnes de ce tableau de données ainsi que sa taille.

In [ ]:
prenoms = pd.read_csv("./prenoms-1900-2023-dpt.csv",sep=";")
print(prenoms.dtypes)
print("Nombre de lignes : {l}, nombre de colonnes : {c}".format(l=prenoms.shape[0],c=prenoms.shape[1]))

Nous pouvons également afficher quelques ligne du jeu de données avec la mèthode head.

In [ ]:
prenoms.head()

L'objet `prenoms` est en effet une [DataFrame pandas](http://pandas.pydata.org/pandas-docs/version/0.23/dsintro.html#dataframe) qui dispose de nombreuse méthodes facilitant la manipulation de données. 

In [ ]:
prenoms.__class__

## Accéder à des valeures : `loc`, `iloc`...

Une des premières choses que nous pouvons faire, c'est accéder à des sous-ensembles de ce jeu de données, sélectionner quelques lignes et colonnnes. 

In [ ]:
# accès par position
prenoms.iloc[125,:]

In [ ]:
# accès par index (nom)
prenoms.loc[:5,'sexe':'annais']

In [ ]:
# selection booleenne
prenoms[prenoms.sexe==2].head()

In [ ]:
# abreviation
prenoms['sexe'].head()

In [ ]:
# abreviation
prenoms.sexe.head()

Chaque colonne d'une DataFrame est une [Serie pandas](https://pandas.pydata.org/pandas-docs/version/0.23.4/api.html#series). Les séries sont des vecteurs unidimensionels avec un index. Elles disposent, elles aussi, de méthodes utiles. 

In [ ]:
prenoms.sexe.__class__

Nous pouvons par exemple utiliser la méthode `unique` pour récupérer la liste des départements de notre jeu de données.

In [ ]:
prenoms.dpt.unique()

## Prétraitement
Nous pouvont remarqué que pour certaines lignes l'année n'est pas renseignée. Il en est de même pour le département. Pour ces lignes les valeurs de l'année est remplacée 'XXXX' et celle du département par 'XX'.

De plus, les prénoms rares ont été regroupés sous l'intitulé '_PRENOMS_RARES', nous allons également éliminer les lignes qui portent cette valeur.

Avant d'analyser les prénoms, nous devons donc éliminer ces lignes du DataFrame.

In [ ]:
# Garder uniquement les lignes qui ne contiennent pas 'XXXX' dans 'annais' ou 'XX' dans 'dpt'
prenoms = prenoms[ ~prenoms['annais'].isin(['XXXX']) & ~prenoms['dpt'].isin(['XX']) & ~prenoms['preusuel'].isin(['_PRENOMS_RARES'])]

# Ré-indexation des lignes après suppression
prenoms.reset_index(inplace=True, drop=True)
prenoms.head(100)

In [ ]:
prenoms = prenoms.astype({'sexe': int, 'annais': int, 'dpt': str})
print(prenoms.dtypes)

## Questions
1. Filtrer les données pour trouver le nombre de Paul nés à paris en 1983 ?<br>Vous pourrer utiliser une sélection booléenne avec plusieurs clauses (attention aux paratenthèses et aux opérateurs utilisés `&`).
2. Combien de naissances sont enregistrées dans ce jeu de données (vous pourrez regarder la méthode [`sum`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.sum.html) ) ?
3. Combien de naissances sont enregistrées à Paris en 1990 ?
4. Dans quel département y a t'il eu le plus de Nathalie en 1983 (vous pourrez regarder la méthode [`sort_values`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html) )?


Dans une série, nous pouvons axcéder aux valeurs (stockées dans un tableau numpy) et à l'index associé.

In [ ]:
prenoms[(prenoms.preusuel=="PAUL") & (prenoms.annais=="1983") & (prenoms.dpt=="75")]

In [ ]:
prenoms.nombre.sum()

In [ ]:
prenoms.loc[(prenoms.dpt==75) & (prenoms.annais==1990),["nombre"]].sum()

In [ ]:
prenoms[(prenoms.annais==1983) & (prenoms.preusuel=="NATHALIE")].sort_values("nombre",ascending=False)

Il peut être intéressant d'avoir accès à ce stockage et utiliser les possibilitée offertes par [numpy](https://docs.scipy.org/doc/numpy/reference/arrays.html). Cela peut, par exemple, nous permettre de cacluler nous même le coefficient d'asymétrie d'une distribution, ou [skewness en anglais](https://fr.wikipedia.org/wiki/Asym%C3%A9trie_(statistiques)), dont la formule est la suivante :

$$skew =  \frac{\sum_{i=1}^N \frac{(x_i-\bar{x})^3}{N}}{s^3}$$

avec $\bar{x}$ la moyenne et $s$ l'écart type.

In [ ]:
n = prenoms.nombre.values

In [ ]:
moyenne = np.mean(n)
ecart_type = np.sqrt(np.mean(np.power(n-moyenne,2)))
L=len(n)

In [ ]:
s3  = np.sum(np.power(n-moyenne,3))/L
s3n = s3/ecart_type**3 
s3n

Nous pouvons vérifier notre résultat en comparant avec celui de la fonction `skew` de la librairie `stats` de `scipy` :

In [ ]:
from scipy.stats import skew
skew(n)

## Gestion des index

Revenons un peu a pandas et parlons des index qui optimisent l'accès et sont également utilisés pour les jointures. **Ils peuvent être multiples**. Lors de la lecture du fichier un index par défaut a été construit avec un simple numéro de ligne. Il est possible de redéfinir nous même nos index avec la méthode `set_index` et en supprimer avec la méthode `reset_index`.

In [ ]:
prenoms_index = prenoms.set_index(['preusuel', 'sexe', 'annais','dpt'])
prenoms_index.head(10)

## Accès à un élément via l'index
Pour en savoir plus sur les [multi-index](https://pandas.pydata.org/pandas-docs/version/0.23.4/advanced.html#advanced).

In [ ]:
prenoms_index.loc[("AARON",1,1985,'75')]

## Résumé statistique
Vous pouvez avoir rapidement une **description statistique** des colonnes numériques d'un DataFrame avec la méthode `describe`. Qui donne les information statistiques élémentaires : moyenne, écart-type, quartile,...

In [ ]:
prenoms.describe()

## Aggrégation : `group_by`
Vous pouvez facilement faire des **opérations de regroupement** avec la méthode `group_by` celle-ci est utilisée ici pour calculer le nombre de naissances de chaque prénom grâce a une simple somme.

In [ ]:
prenoms_total = prenoms[["preusuel","nombre"]].groupby("preusuel").sum().sort_values("nombre",ascending=False)
prenoms_total

Mais vous pouvez fournir une autre opération d'aggrégation, utiliser plusieurs colonnes pour l'aggrégation et enchainer les groupements. La ligne suivante calcule, par exemple, pour chaque prénom et année, le nombre total de naissances et à partir de ce résultat le nombre maximal de naissances observées sur 1 année :

In [ ]:
prenoms[["preusuel","annais","nombre"]].groupby(["preusuel","annais"]).sum().reset_index().groupby("preusuel").agg(np.max).sort_values("nombre",ascending=False)

## Questions

1. Quel département a eu le moins de naissances sur la période 1900-2023 ?
1. Quel prénom a été le plus donné en 1983 ?

In [ ]:
prenoms[["nombre","dpt"]].groupby("dpt").sum().sort_values("nombre")

In [ ]:
prenoms[prenoms.annais==1983][["preusuel","nombre"]].groupby("preusuel").sum().sort_values("nombre",ascending=False).head(1)

## Analyse des prénoms par année
Nous allons étudier un peu les **courbes de popularité des prénoms** de ce jeu de données entre 1900 et 2023 en nous focalisant sur les prénoms les plus fréquents. Pour ce faire nous allons tracer la somme cumulée du pourcentage de naissances suivant le rang des prénoms. Le DataFrame `prenoms_total` est déja trié en ordre décroissant, nous pouvons donc simplement cumuler ces comptages et diviser par le nombre de naissances de manière à nous ramener à un pourcentage :

In [ ]:
cdf = prenoms_total.nombre.cumsum()/prenoms_total.nombre.sum()*100
cdf.head()

Les Marie représentent plus de 2,9 % des naissance dans ce jeu de données et les 5 prénoms les plus courants couvrent près de 8,5 % des naissances.  

In [ ]:
f  = plt.plot(range(0,len(cdf),100),cdf.values[range(0,len(cdf),100)])
xl = plt.xlabel("Rang du prénom")
yl = plt.ylabel("% de naissances cumulées")

Autrement dit, si nous ne conservons que les 1000 prénoms les plus courants, nous couvrirons plus de 94 % des naissances.

In [ ]:
cdf.iloc[999]

Nous allons donc nous focaliser sur ces prénoms et observer l'évolution de leur popularité.

In [ ]:
# récupération de la liste des 1000 prénoms les + utilisés
liste_prenoms_courants = prenoms_total.index.values[:1000]

# filtrage des données pour ne conserver que les prénoms courants, utilisation de la méthode 'isin'
prenoms_freq = prenoms[prenoms.preusuel.isin(liste_prenoms_courants)]

# création de la série temporelle nombre de naissances par année
prenoms_annees = prenoms_freq[["preusuel","annais","nombre"]].groupby(["preusuel","annais"]).sum()
prenoms_annees.head()


Nous pouvons maintenant observer l'évolution temporelle de la popularité de ces prénoms. Regardons, par exemple, les courbes des Josette, Antoine et Emma :

In [ ]:
fig, ax = plt.subplots()
listeprenoms = ["JOSETTE", "EMMA","ANTOINE"]
for p in listeprenoms:
    nbnaiss = prenoms_annees.reset_index("annais").loc[p]
    fj  = ax.plot(nbnaiss.annais,nbnaiss.nombre,label=p)

leg = ax.legend()

## Passage en format large

Méthodes `unstack` et `fillna`.

Nous allons passer les données en format large avec une colonne par année et une ligne par prénom en utilisant la méthode `unstack`. Nous appliquerons également une complétion des données manquantes avec des 0 avec ma méthode `fillna`.

In [ ]:
prenoms_annees_large=prenoms_annees.unstack().fillna(0)["nombre"]
prenoms_annees_large

In [ ]:
years = prenoms_annees_large.columns.values
years

## Analyse par département et année

Création d'une carte animée des **prénoms masculins** les plus donnés par département et par année

In [ ]:
# extraction des prénoms masculin les plus données
prenoms_annees_dep=prenoms_freq.loc[prenoms_freq.sexe==1,].sort_values("nombre", ascending=False).groupby(["annais","dpt"])

In [ ]:
prenoms_annees_dep = prenoms_annees_dep.agg(lambda v: v.iloc[0])

In [ ]:
prenoms_annees_dep

# Web mapping des données

### Recherche des limites des départements
> **Geocoding** avec Nominatim (OSM) : on recherche le nom et les limites d'un département à partir du numéro de ce département<br>

In [ ]:
def get_department_boundaries_with_Nominatim(department_code):
    # Construire l'URL de la requête vers l'API Nominatim
    url = f"https://nominatim.openstreetmap.org/search?state=France&county={department_code}&format=json&polygon_geojson=1&accept-language=fr"
    
    # Utiliser pandas pour lire directement le JSON depuis l'URL
    data = pd.read_json(url)
    
    # Vérifier si des résultats sont trouvés
    if not data.empty:
        # Extraire le GeoJSON (les contours) du DataFrame et le nom du département
        geojson = data['geojson'].iloc[0]  # On suppose qu'il n'y a qu'un seul département pour ce code
        name = data['name'].iloc[0]
        return name, geojson
    else:
        print(f"Aucun département trouvé pour le code {department_code}")
        return None, None

# Exemple d'utilisation pour le département 94 (Val-de-Marne)
department_code = "corse"
nom, geojson = get_department_boundaries_with_Nominatim(department_code)

if nom:
    print(nom)


In [ ]:
import folium

# Créer la carte centrée sur la France
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

# Ajouter le nom et les contours du département au format GeoJSON à la carte
folium.GeoJson(geojson, tooltip=folium.Tooltip(nom)).add_to(m)

# Afficher la carte dans le notebook
m

In [ ]:
# Sauvegarde de la carte au format HTML
m.save("department_map.html")

### Pour l'ensemble des départements présents dans les données

In [ ]:
# Récupération des numéros de départements dans le DataFrame (France métropolitaine)
unique_dpt_values = list(prenoms_annees_dep.index.get_level_values('dpt').unique())
unique_dpt_values = [dpt for dpt in unique_dpt_values if int(dpt) <= 95]
print(unique_dpt_values)

In [ ]:
# Création de la carte et mappage des contours et des noms des départements
from IPython.display import clear_output

# Créer la carte centrée sur la France
m = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

for dep_code in unique_dpt_values:
    # Pour la Corse : 20 est le numéro de la région Corse (2A+2B)
    if dep_code == '20':
        dep_code = 'Corse'
    nom, geojson = get_department_boundaries_with_Nominatim(dep_code)
    print(f"département {dep_code} : {nom}")
    clear_output(wait=True)

    # Ajouter le nom et les contours du département au format GeoJSON à la carte
    folium.GeoJson(geojson, tooltip=folium.Tooltip(f"{nom} ({dep_code})")).add_to(m)

# Afficher la carte dans le notebook
m

In [ ]:
# Sauvegarde de la carte au format HTML
m.save("france_map.html")

In [ ]:
# Récupération des années dans le DataFrame
unique_year_values = list(prenoms_annees_dep.index.get_level_values('annais').unique())
print(unique_year_values)

# A faire

Vous disposez d'une carte des départements et des données annualisées sur la fréquence des prénoms donnés en France.

1. Compléter la carte en reportant sur celle-ci, pour une année donnée, les 5 prénoms masculins les plus fréquents.
2. Faire de même pour les prénoms féminins.
3. Dotter la carte d'un control permettant de sélectionner l'année et le genre (H/F) des prénoms.
   
A vous d'imaginer de quelle manière visualiser ces informations sur la carte (couleurs, marqueurs, ...)